## Node Matching

In [1]:
from utils.node import Node
from utils.ordered_set import OrderedSet
import neonpandas as npd

import pandas as pd

## 1. Utilize the Intersection Method from Custom OrderedSet Class

In [ ]:
a = OrderedSet(('Pet', 'Dog')) # simple test example
a

In [ ]:
b = OrderedSet(('Dog',)) # this should == a
c = OrderedSet('Cat') # this should != a
b, c

In [ ]:
# works just as Set's intersection method
a.intersection(b)

In [ ]:
# simpler, faster adjective-like version
a.has_intersection(b)

In [ ]:
a.has_intersection(c)

## 2. Node Equality: Checks for Intersection of Labels and Matching ID Pattern

In [ ]:
a = Node(('Pet', 'Dog'), 'name', 'Ralph', var='a')
a

In [ ]:
b = Node('Pet', 'name', 'Ralph', var='b')   # should match to a
c = Node('Pet', 'name', 'Pip', var='c')     # should not match to a
d = Node('Owner', 'name', 'Ralph', var='d') # should not match to a (but because of labels)
b, c, d

In [ ]:
for n in [b, c, d]:
    print(a == n)

In [ ]:
for i,j,_expected in [(a, b, True), (a, c, False), (a, d, False)]:
    print(i.var, '==', j.var, '?')
    _result = i == j
    print('expected: {} --- actual: {}'.format(_expected, _result))
    print('passed test: {}'.format((True if _expected == _result else False)))
    print('{}\n'.format('-' * 35))

## 3. Utilized in Node|Edge-Frame Joins

In [136]:
pets = npd.read_csv('pets.csv', id_col='name', lbl_col='species', labels=('Pet'))
pets

,labels,name,color,age,behavior
node,,,,,
"(n:Pet:Dog {name: ""Ralph""})","(Pet, Dog)",Ralph,black,10.0,NaN
"(n:Pet:Cat {name: ""Pip""})","(Pet, Cat)",Pip,yellow,6.0,good
"(n:Pet:Pig {name: ""Babe""})","(Pet, Pig)",Babe,NaN,3.0,NaN
"(n:Pet:Fish {name: ""Bubbles""})","(Pet, Fish)",Bubbles,red,NaN,acceptable
"(n:Pet:Horse {name: ""Freckles""})","(Pet, Horse)",Freckles,brown,NaN,NaN


In [137]:
pets.reset_index(inplace=True)
pets

,node,labels,name,color,age,behavior
0,"(n:Pet:Dog {name: ""Ralph""})","(Pet, Dog)",Ralph,black,10.0,NaN
1,"(n:Pet:Cat {name: ""Pip""})","(Pet, Cat)",Pip,yellow,6.0,good
2,"(n:Pet:Pig {name: ""Babe""})","(Pet, Pig)",Babe,NaN,3.0,NaN
3,"(n:Pet:Fish {name: ""Bubbles""})","(Pet, Fish)",Bubbles,red,NaN,acceptable
4,"(n:Pet:Horse {name: ""Freckles""})","(Pet, Horse)",Freckles,brown,NaN,NaN


In [73]:
# example edges
edges = [
    {'src': 'Ralph', 'rel_type': 'FRIENDLY_WITH', 'dest': 'Bubbles', 'reporter': 'Jenny'},
    {'src': 'Pip', 'rel_type': 'MEAN_TO', 'dest': 'Babe', 'reporter': 'Frank'},
    {'src': 'Ralph', 'rel_type': 'MEAN_TO', 'dest': 'Freckles', 'reporter': 'Frank'},
]
edges = pd.DataFrame(edges)
#edges = npd.EdgeFrame(edges, rel_col='rel_type')
edges

,src,rel_type,dest,reporter
0,Ralph,FRIENDLY_WITH,Bubbles,Jenny
1,Pip,MEAN_TO,Babe,Frank
2,Ralph,MEAN_TO,Freckles,Frank


In [24]:
edges[['rel_type']]

,rel_type,src,dest,reporter,src_node
0,FRIENDLY_WITH,Ralph,Bubbles,Jenny,"(n:Pet {name: ""Ralph""})"
1,MEAN_TO,Pip,Babe,Frank,"(n:Pet {name: ""Pip""})"
2,MEAN_TO,Ralph,Freckles,Frank,"(n:Pet {name: ""Ralph""})"


In [38]:
pets

,node,labels,name,color,age,behavior
0,"(n:Dog {name: ""Ralph""})","(Dog,)",Ralph,black,10.0,NaN
1,"(n:Cat {name: ""Pip""})","(Cat,)",Pip,yellow,6.0,good
2,"(n:Pig {name: ""Babe""})","(Pig,)",Babe,NaN,3.0,NaN
3,"(n:Fish {name: ""Bubbles""})","(Fish,)",Bubbles,red,NaN,acceptable
4,"(n:Horse {name: ""Freckles""})","(Horse,)",Freckles,brown,NaN,NaN


In [41]:
node_col = pets.node.tolist()
node_col[0]

(n:Dog {name: "Ralph"})

In [42]:
edges

,rel_type,src,dest,reporter
0,FRIENDLY_WITH,Ralph,Bubbles,Jenny
1,MEAN_TO,Pip,Babe,Frank
2,MEAN_TO,Ralph,Freckles,Frank


In [43]:
edges['src_node'] = edges.src.apply(lambda x: Node('Pet', 'name', x))
edges

,rel_type,src,dest,reporter,src_node
0,FRIENDLY_WITH,Ralph,Bubbles,Jenny,"(n:Pet {name: ""Ralph""})"
1,MEAN_TO,Pip,Babe,Frank,"(n:Pet {name: ""Pip""})"
2,MEAN_TO,Ralph,Freckles,Frank,"(n:Pet {name: ""Ralph""})"


In [45]:
edges['src_node2'] = edges.src_node.apply(lambda x: [n for n in node_col if n == x][0])
edges

IndexError: list index out of range

In [58]:
set_test = set(node_col)
set_test

{(n:Cat {name: "Pip"}),
 (n:Dog {name: "Ralph"}),
 (n:Fish {name: "Bubbles"}),
 (n:Horse {name: "Freckles"}),
 (n:Pig {name: "Babe"})}

In [64]:
test = Node(('Pet', 'Dog'), 'name', 'Ralph')
test

(n:Pet:Dog {name: "Ralph"})

In [65]:
set_test.intersection({test})

{(n:Pet:Dog {name: "Ralph"})}

In [66]:
set_test & {test}

{(n:Pet:Dog {name: "Ralph"})}

In [ ]:
class Shirt:
    def __init__(self, size, color):
        self.size = size
        self.color = color
        
    def __eq__(self, other_shirt):
        if isinstance(other_shirt, self.__class__):
            return self.size == other_shirt.size and self.color == other_shirt.color
        else:
            return False
        
    def __hash__(self):
        return hash((self.size, self.color))

In [ ]:
my_shirt = Shirt('small', 'blue')
his_shirt = Shirt('small', 'blue')

In [ ]:
my_shirt == his_shirt

In [ ]:
my_shirt.__hash__()

In [ ]:
his_shirt.__hash__()

In [ ]:
tuple({'name': 'ralph'})

In [30]:
a_set = {'dog'}
b_set = {'dog', 'pet'}

In [31]:
a_set == b_set

False

In [98]:
pets = pd.DataFrame([
    {'name': 'Ralph', 'labels': {'Pet', 'Dog'}, 'behavior': 'bad'},
    {'name': 'Pip', 'labels': {'Pet', 'Cat'}, 'behavior': 'good'},
    {'name': 'Bubbles', 'labels': {'Pet', 'Fish'}, 'behavior': 'irrelevant'},
    {'name': 'Freckles', 'labels': {'Pet', 'Horse'}, 'behavior': 'obstinate'}
])
pets['labels'] = pets.labels.apply(lambda x: list(x))
pets

,name,labels,behavior
0,Ralph,"[Dog, Pet]",bad
1,Pip,"[Cat, Pet]",good
2,Bubbles,"[Fish, Pet]",irrelevant
3,Freckles,"[Horse, Pet]",obstinate


In [122]:
edges = pd.DataFrame([
    {'src': 'Ralph', 'rel_type': 'FRIENDLY_WITH', 'dest': 'Bubbles', 'reporter': 'Jenny'},
    {'src': 'Pip', 'rel_type': 'MEAN_TO', 'dest': 'Ralph', 'reporter': 'Frank'},
    {'src': 'Ralph', 'rel_type': 'MEAN_TO', 'dest': 'Freckles', 'reporter': 'Frank'},
])
edges

,src,rel_type,dest,reporter
0,Ralph,FRIENDLY_WITH,Bubbles,Jenny
1,Pip,MEAN_TO,Ralph,Frank
2,Ralph,MEAN_TO,Freckles,Frank


In [107]:
pet_lbls = pets[['name', 'labels']].explode(column='labels')
pet_lbls

,name,labels
0,Ralph,Dog
0,Ralph,Pet
1,Pip,Cat
1,Pip,Pet
2,Bubbles,Fish
2,Bubbles,Pet
3,Freckles,Horse
3,Freckles,Pet


In [112]:
pd.merge(pet_lbls, edges,
        left_on=['labels', 'name'],
        right_on=['src_lbls', 'src'],
        how='inner')

,name,labels,src,rel_type,dest,reporter,src_lbls,dest_lbls
0,Ralph,Pet,Ralph,FRIENDLY_WITH,Bubbles,Jenny,Pet,Pet
1,Ralph,Pet,Ralph,MEAN_TO,Freckles,Frank,Pet,Pet
2,Pip,Pet,Pip,MEAN_TO,Ralph,Frank,Pet,Pet


In [113]:
pets

,name,labels,behavior
0,Ralph,"[Dog, Pet]",bad
1,Pip,"[Cat, Pet]",good
2,Bubbles,"[Fish, Pet]",irrelevant
3,Freckles,"[Horse, Pet]",obstinate


## Broken-Down Loop Way

In [123]:
edges['src_node'] = edges.src.apply(lambda x: Node('Pet', 'name', x))
edges['dest_node'] = edges.dest.apply(lambda x: Node('Pet', 'name', x))
edges.drop(columns=['src', 'dest'], inplace=True)
edges

,rel_type,reporter,src_node,dest_node
0,FRIENDLY_WITH,Jenny,"(n:Pet {name: ""Ralph""})","(n:Pet {name: ""Bubbles""})"
1,MEAN_TO,Frank,"(n:Pet {name: ""Pip""})","(n:Pet {name: ""Ralph""})"
2,MEAN_TO,Frank,"(n:Pet {name: ""Ralph""})","(n:Pet {name: ""Freckles""})"


In [139]:
matches = []
for src in edges.src_node:
    for n in pets.node:
        if src == n:
            matches.append(n)

In [144]:
src_nodes = [find_match(src, pets.node) for src in edges.src_node]

In [147]:
edges['src_nodes2'] = edges.src_node.apply(lambda x: node.find_match(x, pets.node))
edges['dest_nodes2'] = edges.dest_node.apply(lambda x: node.find_match(x, pets.node))
edges

,rel_type,reporter,src_node,dest_node,src_nodes2,dest_nodes2
0,FRIENDLY_WITH,Jenny,"(n:Pet {name: ""Ralph""})","(n:Pet {name: ""Bubbles""})","(n:Pet:Dog {name: ""Ralph""})","(n:Pet:Fish {name: ""Bubbles""})"
1,MEAN_TO,Frank,"(n:Pet {name: ""Pip""})","(n:Pet {name: ""Ralph""})","(n:Pet:Cat {name: ""Pip""})","(n:Pet:Dog {name: ""Ralph""})"
2,MEAN_TO,Frank,"(n:Pet {name: ""Ralph""})","(n:Pet {name: ""Freckles""})","(n:Pet:Dog {name: ""Ralph""})","(n:Pet:Horse {name: ""Freckles""})"
